In [1]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [ ]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

In [ ]:
# gera uma lista de tupla com as possibilidades dos codigos de serventias e um intervalo dos anos
servs_anos = list(itertools.product(list(serventias_jec.cod_serv),range(2013,2017)))

# gera a lista dos arquivos que deverão ser lidos
processos_serv_ano = [TJData.processo(sa[0],sa[1]) for sa in servs_anos]

# colunas que serão utilizadas no processo
usecols=['COD_PROC','COD_COMP','COD_SERV','ID_PROC','COD_ASSUNTO']

# filtra os arquivos inexistentes
#processos_serv_ano = TJData.try_read(processos_serv_ano,usecols=usecols)

In [ ]:
r'{}'.format(TJData.processo('(\d+)','(\d+)'))

In [ ]:
def in_serv_ano(file, file_regex, servs_anos):
    print(tuple(map(lambda x: int(x),re.findall(r'(\d+)',file))) in servs_anos)
    return tuple(map(lambda x: int(x),re.findall(r'(\d+)',file))) in servs_anos
    
    

def list_files_in_serventias_anos(file_regex, servs_anos):
    dirpath = re.search(r'(\w+/)', file_regex).group(0)
    fulldirpath = os.path.join('/tj_files',dirpath)
    onlyfiles = [
        f for f in os.listdir(fulldirpath)
        if os.path.isfile(os.path.join(fulldirpath, f))
        and in_serv_ano(f, file_regex ,servs_anos)
    ]
    return onlyfiles
list_files_in_serventias_anos(TJData.processo('(\d+)','(\d+)'),serventias_jec)

In [ ]:
servs_anos

In [12]:
# Dado a grande quantidade de registros, o primeiro processamento fez com que a maquina travasse, 
# o código abaixo fez com que não precisasse enviar requisições de todos os dados 

# le a tabela original
classes_assuntos = TJData.read_csv(TJData.csv['classe_assunto_2'], usecols=['COD_CLASSE','COD_ASSUNTO'])
classes_assuntos = classes_assuntos.rename(columns={'COD_CLASSE':'classe','COD_ASSUNTO':'assunto'})

# obtem os registros no banco
ca = api.get('classesassuntos')
ca = pd.DataFrame(api.response_to_json(ca)).drop(columns=['id'])

# cria conjuntos Set e obtem a diferença dos dados não cadastrados
in_db = set(map(lambda x: (x['classe'],x['assunto']),TJData.to_json(ca)))
original = set(map(lambda x: (x['classe'],x['assunto']),TJData.to_json(classes_assuntos)))
print(f'in_db: {len(in_db)}',f'original: {len(original)}')
out_db = pd.DataFrame(original.difference(in_db),columns=['classe','assunto'])
print(f'out_db: {len(out_db)}')
out_db = TJData.to_json(out_db)

# filtra as pks de assunto inexistentes
#{'{"assunto":["Invalid pk \\"30521\\" - object does not exist."]}',
# '{"assunto":["Invalid pk \\"30523\\" - object does not exist."]}'}
out_db = list(filter(lambda x: x['assunto'] not in [30521,30523], out_db))

# divide em listas de 5000 registros por request
out_db_splitted = api.split_list(out_db, len(out_db)//5000)

in_deb: 556749 original: 590392
out_db: 33643


In [ ]:
r_post = api.concurrent_request(api.post,'classesassuntos',out_db_splitted)

In [ ]:
out_db_splitted = api.split_list(out_db, len(out_db)//1000)

In [8]:
len(out_db_splitted)

19

In [13]:
r_post

[([{'classe': 1646, 'assunto': 11057},
   {'classe': 418, 'assunto': 11238},
   {'classe': 1595, 'assunto': 3498},
   {'classe': 1602, 'assunto': 11347},
   {'classe': 1628, 'assunto': 3537},
   {'classe': 326, 'assunto': 11357},
   {'classe': 1581, 'assunto': 9748},
   {'classe': 1560, 'assunto': 30135},
   {'classe': 1632, 'assunto': 9879},
   {'classe': 375, 'assunto': 30163},
   {'classe': 323, 'assunto': 9728},
   {'classe': 326, 'assunto': 9911},
   {'classe': 1690, 'assunto': 9819},
   {'classe': 408, 'assunto': 30058},
   {'classe': 1727, 'assunto': 11178},
   {'classe': 1729, 'assunto': 30497},
   {'classe': 1650, 'assunto': 11253},
   {'classe': 1620, 'assunto': 11342},
   {'classe': 1658, 'assunto': 11272},
   {'classe': 322, 'assunto': 9670},
   {'classe': 10944, 'assunto': 11248},
   {'classe': 1612, 'assunto': 11202},
   {'classe': 1598, 'assunto': 10608},
   {'classe': 333, 'assunto': 11138},
   {'classe': 238, 'assunto': 2620},
   {'classe': 1611, 'assunto': 11196},
   